1.6.1: Functions as Arguments

In [2]:
def summation(n, func):
    k, res = 1, 0
    while k <= n:
        res, k = res + func(k), k + 1
    return res

def cube(x):
    return x*x*x

def identity(x):
    return x

def pi_term(x):
    return 8/((4 * x - 3) * (4 * x - 1))

def sum_cubes(n):
    return summation(n, cube)

def sum_naturals(n):
    return summation(n, identity)

def pi_sum(n):
    return summation(n, pi_term)

In [ ]:
sum_cubes(10)

In [ ]:
sum_naturals(10)

In [ ]:
pi_sum(1000)

1.6.2 Functions as General Methods

In [ ]:
def improve(close, update, guess = 1):
    while not close(guess):
        guess = update(guess)
    return guess

In [ ]:
def golden_update(guess):
    return 1/guess + 1

def golden_close(guess):
    return approx_eq(guess * guess, guess + 1)

def approx_eq(x, y, threshold = 1e-15):
    return abs(x - y) < threshold

In [ ]:
improve(golden_close, golden_update, -100000000)

1.6.3: Defining Functions III: Nested Definitions

In [ ]:
def sqrt(a):
    def sqrt_update(x):
        return (x + a/x) / 2
    def sqrt_close(x):
        return approx_eq(x * x, a)
    return improve(sqrt_close, sqrt_update)

In [ ]:
sqrt(9)

1.6.4: Functions as Returned Values

In [ ]:
""" def compose1(f, g):
    def h(x):
        return f(g(x))
    return h """

In [ ]:
def square(x):
    return x * x

def successor(x):
    return x + 1

In [ ]:
successor_squared = compose1(square, successor)
successor_squared(12)

In [ ]:
def make_adder(f):
    def adder(k):
        return f + k
    return adder

In [ ]:
add3 = make_adder(3)
add3(2) 

1.6.5: Example Newton's Method

In [ ]:
def newton_update(f, df):
    def update(x):
        return x - f(x)/df(x)
    return update

def find_zero(f, df):
    def near_zero(x):
        return approx_eq(f(x), 0)
    return improve(near_zero, newton_update(f, df))

In [ ]:
def square_root_newton(a):
    def f(x):
        return x * x - a
    def df(x):
        return 2 * x
    return find_zero(f, df)

In [ ]:
square_root_newton(64)

In [ ]:
def power(x, n):
    product, k = 1, 0
    while k < n:
        product, k = product * x, k + 1
    return product

In [ ]:
def nth_root_newton(a, n):
    def f(x):
        return power(x, n) - a
    def df(x):
        return n * power(x, n - 1)
    return find_zero(f, df)

In [ ]:
nth_root_newton(100, 2)

1.6.6: Currying

In [ ]:
def curried_pow(x):
    def h(y):
        return pow(x, y)
    return h


In [ ]:
curried_pow(5)(5)

In [ ]:
def map_to_range(start, end, f):
    while start < end:
        print(f(start))
        start += 1

In [ ]:
map_to_range(0, 10, curried_pow(5))

In [ ]:
def curry2(f):
    """Return a curried version of the given two-argument function."""
    def g(x):
        def h(y):
            return f(x, y)
        return h
    return g

def uncurry2(g):
    """Return a two-argument version of the given curried function."""
    def f(x, y):
        return g(x)(y) 
    return f

In [ ]:
pow_curried = curry2(pow)
pow_curried(5)(6)

In [ ]:
uncurry2(pow_curried)(5, 6)

1.6.7: Lambda Expressions

In [ ]:
def compose1(f, g):
    return lambda x: f(g(x))

"""compose1 = lambda f,g: lambda x: f(g(x))"""

In [ ]:
s = lambda x: x * x
s(12)

1.6.9: Function Decorators

In [ ]:
def trace(fn):
        def wrapped(x):
            print('-> ', fn, '(', x, ')')
            return fn(x)
        return wrapped

In [ ]:
h = trace(pow_curried)
h(50)(2)

In [ ]:
@trace
def triple(x):
    """The @ decorator affects the def statement, triple is not bound to this function.
    Insted the name triple is bound to the returned function value of calling trace
    on the newly defined triple function. It is equivalent to 
    def triple(x):
        return x * 3
    triple = trace(triple)"""
    return 3 * x

In [ ]:
triple(5)

Lecture 6: Iteration

Inverse Function

In [ ]:
def search(f):
    x = 0
    while not f(x):
        x += 1
    return x

def inverse(f):
    """Return g(y) such that g(f(x)) = x."""
    return lambda y: search(lambda x: f(x) == y)

def square(x):
    return x * x

sqrt = inverse(square)
sqrt(256)

Self-reference

In [ ]:
def print_sum(x):
    print(x)
    def next_sum(y):
        return print_sum(x + y)
    return next_sum

print_sum(1)(3)(5)

In [ ]:
def isPalindrome(s):
    return s == s[::-1]

isPalindrome("liil")

1.7: Recursive Functions

In [ ]:
#Sum of digits
def sum_digits(n):
    if n < 10:
        return n
    last, all_but_last = n % 10, n // 10
    return last + sum_digits(all_but_last)

In [ ]:
#Factorial
def fact(n):
    if n == 1:
        return 1
    else:
        return n * fact(n - 1)
fact(10)

1.7.2: Mutual Recursion

In [ ]:
def is_even(n):
    if n == 0:
        return True
    else:
        return is_odd(n - 1)

def is_odd(n):
    if n == 0:
        return False
    else:
        return is_even(n - 1)

In [ ]:
def cascade(n):
    """"Print a cascade of prefixes of n"""
    if n < 10:
        print(n)
    else:
        print(n)
        cascade(n // 10)
        print(n)
cascade(20134)

In [ ]:
def play_alice(n):
    if n == 0:
        print("Bob wins")
    else:
        play_bob(n - 1)

def play_bob(n):
    if n == 0:
        print("Alice wins")
    else:
        if is_even(n):
            play_alice(n - 2)
        else:
            play_alice(n - 1)
play_alice(20)

1.7.4: Tree Recursion

In [ ]:
def fib(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

1.7.5: Example Partitions

In [ ]:
def count_partitions(n, m):
    if n == 0:
        return 1
    elif n < 0:
        return 0
    elif m == 0:
        return 0
    else:
        return count_partitions(n - m, m) + count_partitions(n, m - 1)

count_partitions(6,4)

Lecture 9: Recursion

In [ ]:
#Luhn algorithm
def split(n):
    return n % 10, n // 10

def luhn_sum_iter(n):
    k, temp, total = 1, n, 0
    while temp > 0:
        last, all_but_last = split(temp)
        temp = all_but_last
        if k % 2 == 1:
            if last * 2 > 9:
                total += sum_digits(last * 2)
            else:
                total += last * 2
        else:
            total += last
        k += 1
    return total

def luhn_sum_recursion(n):
    if n < 10:
        return n
    else:
        last, all_but_last = split(n)
        return luhn_sum_double(all_but_last) + last

def luhn_sum_double(n):
    if n < 5:
        return n * 2
    else:
        last, all_but_last = split(n)
        return sum_digits(last * 2) + luhn_sum_recursion(all_but_last)

def luhn_check(n):
    if n % 10 == 0:
        return True
    else:
        return False
luhn_sum_recursion(138743)


Lecture 10: Tree Recursion

In [3]:
def inverse_cascade(n):
    def f_then_g(f, g, n):
        if n:
            f(n)
            g(n)
    grow = lambda n: f_then_g(grow, print, n // 10)
    shrink = lambda n: f_then_g(print, shrink, n // 10)
    grow(n)
    print(n)
    shrink(n)
inverse_cascade(1234)

1
12
123
1234
123
12
1


In [1]:
def near_largest_change(num):
    i = 0
    while num != 1:
        num //= 2
        i += 1
    return i
near_largest_change(100)

6